In [1]:
import numpy as np
import cv2
import moviepy.editor as mpy_
from skimage.io import imread_collection
from scipy import signal
import moviepy.video.io.ImageSequenceClip as m


In [4]:
def find_of(Image1, Image2):
    Image1_gray = cv2.cvtColor(Image1,cv2.COLOR_BGR2GRAY)
    Image2_gray = cv2.cvtColor(Image2,cv2.COLOR_BGR2GRAY)
    
    I1 = np.array(Image1_gray)
    I2 = np.array(Image2_gray)
    
    S = np.shape(I1)

    I1_smooth = cv2.GaussianBlur(I1 ,(7,7),1)
    I2_smooth = cv2.GaussianBlur(I2 ,(7,7),1)
    
    Ix = signal.convolve2d(I1_smooth,[[-0.5,0.5],[-0.5,0.5]],'same') 
    Iy = signal.convolve2d(I1_smooth,[[-0.5,-0.5],[0.5,0.5]],'same') 
    It = signal.convolve2d(I1_smooth,[[0.5,0.5],[0.5,0.5]],'same') + signal.convolve2d(I2_smooth,[[-0.5,-0.5],[-0.5,-0.5]],'same')

    features = cv2.goodFeaturesToTrack(I1_smooth ,50 ,0.01,10)
    feature = np.int0(features)
    u = v = np.nan*np.ones(S)
    for l in feature:
        j,i = l.flatten()
        IX = ([Ix[i-1,j-1],Ix[i,j-1],Ix[i-1,j-1],Ix[i-1,j],Ix[i,j],Ix[i+1,j],Ix[i-1,j+1],Ix[i,j+1],Ix[i+1,j-1]]) 
        IY = ([Iy[i-1,j-1],Iy[i,j-1],Iy[i-1,j-1],Iy[i-1,j],Iy[i,j],Iy[i+1,j],Iy[i-1,j+1],Iy[i,j+1],Iy[i+1,j-1]]) 
        IT = ([It[i-1,j-1],It[i,j-1],It[i-1,j-1],It[i-1,j],It[i,j],It[i+1,j],It[i-1,j+1],It[i,j+1],It[i+1,j-1]]) 
        
        LK = (IX, IY)
        LK = np.array(np.matrix(LK))
        LK_tr = np.array(LK.T) 

        AtA = np.dot(LK,LK_tr) 
        AtA_inv = np.linalg.pinv(AtA)
        Ab = np.dot(AtA_inv,LK)

        (u[i,j],v[i,j]) = np.dot(Ab,IT) 
        
    image=Image1.copy()
    for i in range(S[0]):
        for j in range(S[1]):
            if abs(u[i,j])>t or abs(v[i,j])>t: # setting the threshold 
                startpoint=(int(j),int(i))
                endpoint=(j +abs(int(v[i,j]*5)) ,i-(int(u[i,j]*5)))
                color=(0, 255, 0) 
                thickness=3
                image = cv2.arrowedLine(Image1, startpoint,endpoint, color,thickness)
    return image

t = 0.3 # threshold value

images_list = []
col_dir = 'DJI_0101/*.png'
frames = imread_collection(col_dir)

for i in range(0,460,4):
    Image1 = frames[i]
    Image2 = frames[i+3]
    result=find_of(Image1, Image2)
    images_list.append(result)


In [5]:
clip = m.ImageSequenceClip(images_list, fps = 20)
clip.write_videofile("hw4_part1_video.mp4", codec="libx264")

t:   2%|█▏                                                                   | 2/115 [00:00<00:08, 13.48it/s, now=None]

Moviepy - Building video hw4_part1_video.mp4.
Moviepy - Writing video hw4_part1_video.mp4



Moviepy - Done !
Moviepy - video ready hw4_part1_video.mp4


In [6]:
images_list_2 = []
col_dir = 'DJI_0101/*.png'
frames = imread_collection(col_dir)
background = frames[459]
back_smooth = cv2.GaussianBlur(background,(21,21),0)

for i in range(0,459):
    Image1 = frames[i]
    Image1_smooth = cv2.GaussianBlur(Image1,(21,21),0)
    difference = cv2.absdiff(back_smooth, Image1_smooth)
    images_list_2.append(difference)


In [7]:
clip = m.ImageSequenceClip(images_list_2, fps = 20)
clip.write_videofile("hw4_part2_video.mp4", codec="libx264")

t:   5%|███▍                                                               | 24/460 [00:00<00:01, 239.76it/s, now=None]

Moviepy - Building video hw4_part2_video.mp4.
Moviepy - Writing video hw4_part2_video.mp4



Moviepy - Done !
Moviepy - video ready hw4_part2_video.mp4
